In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import datetime
import glob ,sys , re
from multiprocessing import Pool
import multiprocessing
from categorical_encoding import select_category_value_agg, select_level_agg
from mv_wg_avg import exp_weight_avg

sys.path.append('../../../github/module/')
from preprocessing import set_validation, split_dataset, get_dummies, factorize_categoricals
from convinience_function import get_categorical_features, get_numeric_features, row_number, pararell_process
from logger import logger_func
from feature_engineering import base_aggregation, diff_feature, division_feature
from make_file import make_npy
from tqdm import tqdm

logger = logger_func()
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 101)

unique_id = 'SK_ID_CURR'
p_id = 'SK_ID_PREV'
target = 'TARGET'
ignore_features = [unique_id, target, 'valid_no', 'is_train', 'is_test', 'SK_ID_PREV']

2018-07-13 14:29:25,036 logger 19 [INFO]    [logger_func] start 
2018-07-13 14:29:25,054 logger 19 [INFO]    [logger_func] start 
2018-07-13 14:29:25,054 logger 19 [INFO]    [logger_func] start 


In [2]:
base = pd.read_csv('../data/base.csv')[unique_id].to_frame()
prev = pd.read_csv('../data/previous_cleansing.csv')
app = pd.read_csv('../data/application_train_test.csv')
birth = app['DAYS_BIRTH'].values
birth = birth - birth.min() +1
birth = np.log10(birth)

In [53]:
app_list = [unique_id, 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'AMT_GOODS_PRICE']
same_list = [unique_id, 'NAME_CONTRACT_STATUS', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'DAYS_DECISION', 'DAYS_LAST_DUE']

app = app[app_list]
app['DAYS_DECISION'] = 0
app['DAYS_LAST_DUE'] = np.nan
app['DAYS_BIRTH_log10'] = birth
app['NAME_CONTRACT_STATUS'] = 'Approved'
tmp_app = app[same_list]
prev = prev[same_list]
aprev = pd.concat([tmp_app, prev], axis=0)
aprev = aprev.merge(app[[unique_id, 'AMT_INCOME_TOTAL', 'DAYS_BIRTH']], on=unique_id, how='inner')
aprev['DAYS_BIRTH'] = aprev['DAYS_BIRTH'] - (aprev['DAYS_DECISION']/12).astype('int')
# birth_min = aprev['DAYS_BIRTH'].min() +1
aprev['DAYS_BIRTH_log10'] = aprev['DAYS_BIRTH'].map(lambda x: np.log10(np.abs(x)))

aprev.sort_values(by=[unique_id, 'DAYS_DECISION'], ascending=False, inplace=True)
aprev['lag_DECISION'] = aprev[[unique_id, 'DAYS_DECISION']].groupby(unique_id).shift(-1)
aprev['DAYS_DECISION_diff@'] = (aprev['DAYS_DECISION'] - aprev['lag_DECISION']).fillna(0)

In [57]:
decision_diff_mean = aprev.groupby(unique_id, as_index=False)['DAYS_DECISION_diff@'].mean()
decision_diff_std = aprev.groupby(unique_id, as_index=False)['DAYS_DECISION_diff@'].std()
prefix='ap_'
result = base.merge(decision_diff_mean, on=unique_id, how='left')
result = base.merge(decision_diff_std, on=unique_id, how='left')
make_npy(result=result, ignore_list=ignore_features, logger=logger, prefix=prefix)

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:970: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  'representation', UserWarning)
2018-07-13 15:09:19,765 logger 29 [INFO]    [make_npy] Series([], Name: DAYS_DECISION_diff@, dtype: int64) 
2018-07-13 15:09:19,765 logger 29 [INFO]    [make_npy] Series([], Name: DAYS_DECISION_diff@, dtype: int64) 


In [58]:
' Approveされたデータ '
apr = aprev.query("NAME_CONTRACT_STATUS=='Approved'")
ref = aprev.query("NAME_CONTRACT_STATUS=='Refused'")
dataset = apr

' credit / application '
dataset = division_feature(df=dataset, first='AMT_CREDIT', second='AMT_GOODS_PRICE')
' credit / annuity '
dataset['cnt_annuity'] = dataset['AMT_CREDIT'] / dataset['AMT_ANNUITY']
' credit / application '
dataset = division_feature(df=dataset, first='AMT_GOODS_PRICE', second='AMT_ANNUITY')
dataset = division_feature(df=dataset, first='AMT_CREDIT', second='DAYS_BIRTH')
dataset = division_feature(df=dataset, first='AMT_ANNUITY', second='DAYS_BIRTH')

for col in ['AMT_CREDIT', 'AMT_GOODS_PRICE', 'AMT_ANNUITY']:
    dataset = division_feature(df=dataset, first=col, second='AMT_INCOME_TOTAL')

SystemExit: 

/mnt/c/Users/gixo/go/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [37]:
apr.head()

,SK_ID_CURR,NAME_CONTRACT_STATUS,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,DAYS_DECISION,DAYS_LAST_DUE,AMT_INCOME_TOTAL,DAYS_BIRTH,DAYS_BIRTH_log10,lag_DECISION,DAYS_DECISION_diff@
0,100002,Approved,406597.5,24700.500,351000.0,0.0,NaN,202500.0,-315,2.498311,NaN,0.0
1,100002,Approved,179055.0,9251.775,179055.0,-20.0,0.0,202500.0,-314,2.496930,0.0,20.0
2,100003,Approved,1293502.5,35698.500,1129500.0,0.0,NaN,270000.0,-558,2.746634,NaN,0.0
3,100003,Approved,68053.5,6737.310,68809.5,-78.0,-66.0,270000.0,-552,2.741939,0.0,78.0
4,100003,Approved,348637.5,44946.990,337500.0,-27.0,-21.0,270000.0,-556,2.745075,-78.0,-51.0


In [61]:
prefix = 'ap_'
dataset = aprev
' 重み付き平均 '
weight_list = [0.99,0.95, 0.9]
# feature_list = [col for col in apr.columns if col.count('AMT_') or col.count('annuity')]
feature_list = ['DAYS_DECISION_diff@']

for num in feature_list:
    for weight in weight_list:
        prev_wavg = exp_weight_avg(data=dataset, level=unique_id, weight=weight, label='DAYS_DECISION', value=num)
        result = base.merge(prev_wavg.to_frame().reset_index(), on=unique_id, how='left').fillna(0)
        make_npy(logger=logger, result=result, ignore_list=ignore_features, prefix=prefix)

2018-07-13 15:09:57,268 logger 29 [INFO]    [make_npy] 0.000000     17700
55.266817     4857
5.803713      2227
5.251047      2156
4.703381      2151
6.361374      1959
6.924024      1892
7.491656      1885
8.064263      1882
8.641839      1871
9.224375      1815
4.160719      1754
9.811864      1695
10.404297     1689
11.001663     1523
11.603955     1445
12.823270     1358
12.211160     1358
3.623066      1348
13.440272     1273
42.215325     1201
2.562801      1196
3.090426      1159
14.688907     1086
14.062155     1058
15.320516      981
15.956968      882
16.598250      867
17.244349      762
18.550936      719
17.895249      681
19.211395      660
2.040196       611
19.876610      606
20.546566      582
21.221245      578
21.900630      528
3.560285       520
2.811467       491
3.183548       476
3.941697       465
22.584705      388
34.150532      364
2.444023       343
31.152530      330
4.327803       327
30.413677      326
23.273452      307
31.895812      304
52.695622     

2018-07-13 15:09:59,463 logger 29 [INFO]    [make_npy] 0.000000     17700
79.982526     4857
8.372592      2205
6.486858      2187
7.414666      2156
9.357239      1977
10.365284     1892
11.393527     1885
12.438934     1876
13.498669     1874
14.570115     1815
5.592578      1716
15.650886     1695
16.738832     1689
17.832040     1522
18.928827     1446
21.127484     1358
20.027728     1358
4.735179      1348
22.227029     1277
79.965059     1201
3.143685      1196
3.917879      1159
24.422070     1086
23.325469     1061
25.516238      981
26.607503      882
27.695506      867
28.779983      762
30.937697      719
29.860751      681
32.010766      660
2.415313       611
33.079953      606
34.145290      582
35.206846      578
36.264713      528
5.891513       520
4.298916       493
5.070877       477
6.758329       465
37.319003      388
79.947600      364
3.577646       343
49.743633      330
7.668422       327
48.721003      326
38.369844      307
50.764534      304
76.976931     

In [70]:
halfyear = aprev.query('DAYS_DECISION >= -6')
half_apr = halfyear.query("NAME_CONTRACT_STATUS=='Approved'")
half_ref = halfyear.query("NAME_CONTRACT_STATUS=='Refused'")

data = halfyear
data = data.groupby(unique_id).size().reset_index().rename(columns={0:'prev_cnt@'})

prefix='ap_HALF_'
result = base.merge(data, on=unique_id, how='left')
make_npy(result=result, ignore_list=ignore_features, logger=logger, prefix=prefix)

2018-07-13 15:18:20,694 logger 29 [INFO]    [make_npy] 1     236304
2      57446
3      28374
4      14079
5       7767
6       4516
7       2647
8       1637
9       1078
10       701
11       486
12       348
13       233
14       158
15       108
16        68
17        66
18        49
19        48
20        28
22        21
21        16
23        13
24        10
25        10
26         7
27         6
32         4
29         4
31         4
28         3
33         3
37         2
42         2
35         2
44         1
38         1
47         1
36         1
34         1
30         1
63         1
Name: prev_cnt@, dtype: int64 
2018-07-13 15:18:20,694 logger 29 [INFO]    [make_npy] 1     236304
2      57446
3      28374
4      14079
5       7767
6       4516
7       2647
8       1637
9       1078
10       701
11       486
12       348
13       233
14       158
15       108
16        68
17        66
18        49
19        48
20        28
22        21
21        16
23        13
24        10
2

In [ ]:
def extract_same_during_annuity(row):

    remaing = row_list.copy()
    remaing.remove(row)
    
    ''' 
    tmp_resultは、ループで回ってきたrow_noを持ったレコードに絞ったDF。
    この後のremaing_loopでこのレコードと同じ期間を持ったレコードを絞り、UNIONしていく 
    '''
    tmp_result = bureau.query(f'row_no=={row}')[[unique_id, b_id, 'row_no']]
    tmp = bureau.query(f'row_no=={row}')[[unique_id, b_id, 'row_no', 'DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE']].rename(columns={'DAYS_CREDIT':'start', 'DAYS_CREDIT_ENDDATE':'end'})
    
    for rem in tqdm(remaing):
        tmp2 = bureau.query(f'row_no=={rem}')[[unique_id, b_id, 'row_no', 'DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE']].rename(columns={'DAYS_CREDIT':'start2', 'DAYS_CREDIT_ENDDATE':'end2', 'row_no':'same_term_row_no', b_id:'add_id'})
        
        ' rowの期間とremaingで回ってきた他の行の期間を比較する為、merge。 '
        df = tmp2.merge(tmp, on=unique_id, how='left')
        
        start = df['start'].values
        end = df['end'].values
        start2 = df['start2']
        end2 = df['end2']
        
        start2 = start2[start <= start2][start2 <= end].to_frame().reset_index()['index'].to_frame()
        end2 = end2[start <= end2][end2 <= end].to_frame().reset_index()['index'].to_frame()
        index = pd.concat([start2, end2], axis=0).drop_duplicates()
        index['include_flg'] = 1
        df.reset_index(inplace=True)
        same_term = df.merge(index, on='index', how='inner')[[unique_id, 'add_id', 'same_term_row_no']].rename(columns={'add_id':b_id, 'same_term_row_no':'row_no'})
        
        tmp_result = pd.concat([tmp_result, same_term], axis=0)
        
    # ここでbureauにmergeするのは、AMT_CREDITなどをつけるため
    target = bureau.merge(tmp_result, on=[unique_id, b_id, 'row_no'], how='inner')
    target['serial_no'] = row
    print(row)
    return target

In [ ]:
def pararell_same_during_annuity(base, app, bureau):
    '''
    同時期に支払いの生じていたbureau_idをまとめ、月あたり支払額の合計を作る。
    延滞を起こしたbureauについて、同時期に支払っていた金額を求める。また、それをINCOMEで割る。
    同時期に無理な割合の支払額を持っていたのではないか？という仮説
    '''
    
    value = 'LOAN_MONTHLY'
#     value = 'PAYMENT_MONTHLY'
    ' DAYS_CREDIT_ENDDATEがNaNの場合、仮決めで3ヶ月後を完了日とする '
    bureau['ENDDATE_CREDIT'] = bureau['ENDDATE_CREDIT'].fillna(3)
    
    p_list = pararell_process(extract_same_during_annuity, row_list)
    
    # 同時期のセットが3つとすると、それが3回ループでやってくるが、重み付き平均チックなのが良いので3つとも積んで平均とる
    print(len(p_list))
    result = pd.concat(p_list, axis=0)
    serial = result['serial_no'].min()
    
    result.to_csv(f'../data/csv_feature/{value}_same_term_feature_{serial}.csv', index=False)
    
# row_list = list(bureau['row_no'].drop_duplicates())
row_list = list(range(1,11,1))
# row_list = list(range(11,78,1))
# pararell_same_during_annuity(base, app, bureau)

a = pd.read_csv('../data/csv_feature/LOAN_MONTHLY_same_term_feature_1.csv')
b = pd.read_csv('../data/csv_feature/LOAN_MONTHLY_same_term_feature_11.csv')
result = pd.concat([a,b], axis=0)
result.to_csv(f'../data/csv_feature/LOAN_MONTHLY_same_during_feature.csv', index=False)